# Player Search
This notebook explores the use of these two endpoints in retrieving individual NBA player statstics:
- https://v2.nba.api-sports.io/players
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics import app

import pandas as pd
import requests

In [2]:
base_url = app.config["SPORTS_API_URL"]
headers = {"x-rapidapi-key": app.config["SPORTS_API_KEY"]}

### Get Player Info from Name

In [3]:
PLAYER_COLUMNS = [
    "id",
    "firstname",
    "lastname",
    "birth_date",
    "country",
    "height_feet",
    "height_inches",
    "weight_pounds",
    "jersey",
    "is_active",
    "position",
    "start_year",
    "pro_years",
    "college",
]

In [4]:
def get_raw_player_data(name):
    url = f"{base_url}players"
    params = {"name": name}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [5]:
def filter_nba_players(players):
    players["is_nba"] = players["leagues"].apply(lambda obj: "standard" in obj.keys())
    return players.loc[players.is_nba]

In [6]:
def clean_player_data(data):
    data["birth_date"] = data["birth"].apply(lambda obj: obj.get("date"))
    data["country"] = data["birth"].apply(lambda obj: obj.get("country"))
    data["start_year"] = data["nba"].apply(lambda obj: obj.get("start"))
    data["pro_years"] = data["nba"].apply(lambda obj: obj.get("pro"))
    data["height_feet"] = data["height"].apply(lambda obj: obj.get("feets"))
    data["height_inches"] = data["height"].apply(lambda obj: obj.get("inches"))
    data["weight_pounds"] = data["weight"].apply(lambda obj: obj.get("pounds"))
    data["jersey"] = data["leagues"].apply(lambda obj: obj["standard"].get("jersey"))
    data["is_active"] = data["leagues"].apply(lambda obj: obj["standard"].get("active"))
    data["position"] = data["leagues"].apply(lambda obj: obj["standard"].get("pos"))

    return data[PLAYER_COLUMNS]

In [7]:
data = get_raw_player_data(name="curry")
if data["results"] == 0:
    print("No player matches")

df_results = pd.DataFrame(data["response"])
df_results = filter_nba_players(players=df_results)
df_results = clean_player_data(data=df_results)

In [8]:
df_results

,id,firstname,lastname,birth_date,country,height_feet,height_inches,weight_pounds,jersey,is_active,position,start_year,pro_years,college
0,123,Seth,Curry,1990-08-23,USA,6,2,185,30,True,G,2013,7,Duke
1,124,Stephen,Curry,1988-03-14,USA,6,2,185,30,True,G,2009,12,Davidson


### Get Player Stats from ID